In this seminar, you've explored a basic implementation of the Deep Structured Semantic Model (DSSM).

Your task is to **improve this model** in one or more of the following directions:

### ✅ Model Improvements
- [ ] Replace MLP towers with Transformer or RNN encoders or etc. (5 баллов)
- [x] Use different triplet loss. (3 балла)
- [x] Add dropout, batch normalization, or layer norm. (3 балла)
- [x] Integrate embeddings instead of one-hot vectors. (5 баллов)
- [ ] Visualize similarity distribution for positive vs. negative pairs. (5 баллов)

### ✅ Evaluation & Analysis
- [x] Visualize embeddings using t-SNE or UMAP. (3 баллов)
- [x] Develop and improve beyond accuracy metrics. (5 баллов)

### 📄 Deliverables
- [x] Explain what you changed and why in the final markdown cell. (3 балла)
- [x] Keep code modular, clean, and well-documented. (3 балла)

### 📝 Production
- create service based on DSSM vectors with ANN. (8 баллов)

### 📝 Leaderboard
- Improve score from UserKNN via DSSM (8 баллов)


Максимум баллов, которые можно получить - 25.

In [1]:
# ----------------------
# 2. IMPORTS AND SETUP
# ----------------------
import os
import requests
import zipfile
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.preprocessing import OneHotEncoder
from collections import Counter
import warnings
import umap
warnings.filterwarnings("ignore")

/home/evgenii-iurin/work/ITMO-RecSys/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# ----------------------
# 3. DOWNLOAD AND LOAD DATA
# ----------------------
def download_and_extract():
    url = 'https://github.com/irsafilo/KION_DATASET/raw/f69775be31fa5779907cf0a92ddedb70037fb5ae/data_original.zip'
    filename = 'kion_train.zip'

    response = requests.get(url, stream=True)
    with open(filename, 'wb') as f:
        total = int(response.headers.get('content-length', 0))
        progress = tqdm(response.iter_content(1024 * 1024),
                        f"Downloading {filename}",
                        total=total // (1024 * 1024), unit='MB')
        for chunk in progress:
            f.write(chunk)

    with zipfile.ZipFile(filename, 'r') as zip_ref:
        zip_ref.extractall("data")
    os.remove(filename)

if not os.path.exists("data/data_original"):
    download_and_extract()



In [3]:
# ----------------------
# 4. DATA PREPROCESSING
# ----------------------
interactions_df = pd.read_csv('data/data_original/interactions.csv', parse_dates=["last_watch_dt"])
users_df = pd.read_csv('data/data_original/users.csv')
items_df = pd.read_csv('data/data_original/items.csv')

# Label Encoding
Вместо использования one-hot векторов закодируем каждую фичу

In [4]:
# Input:
# - user_df - исходный датафрейм с юзерами

# Oupput:
# - user_df - отфильтрованный датафрейм

# Преобразования:
# - Фильтруем не нужные фичи в датафрейме

from sklearn.preprocessing import LabelEncoder

# Заполняем строки "unknown"
for col in ['income', 'sex', 'age']:
    users_df[col] = users_df[col].fillna('unknown')

USERS_FEATURES = ["age", "income", "sex", "kids_flg"]
users_df = users_df[USERS_FEATURES + ["user_id"]]

# Encoding
user_encoders = {}
for feat in USERS_FEATURES:
    le = LabelEncoder()
    users_df[feat] = le.fit_transform(users_df[feat])
    user_encoders[feat] = le

users_df.head()

,age,income,sex,kids_flg,user_id
0,1,4,2,1,973171
1,0,2,2,0,962099
2,3,3,1,0,1047345
3,3,2,1,0,721985
4,2,4,1,0,704055


In [5]:
# Input:
# - items_df - исходный датафрейм с айтемами

# Oupput:
# - items_df - отфильтрованный датафрейм

# Преобразования:
# - Фильтруем не нужные фичи в датафрейме
# - Избавляемся от NaN значений

ITEMS_FEATURES = ['content_type', 'release_year', 'for_kids', 'age_rating', 'studios', 'countries', 'directors']

# Заполняем строки "unknown"
for col in ['content_type', 'studios', 'countries', 'directors']:
    items_df[col] = items_df[col].fillna('unknown')

# Заполняем числовые фичи специальным значением (-1)
for col in ['release_year', 'for_kids', 'age_rating']:
    items_df[col] = items_df[col].fillna(-1)

items_df = items_df[ITEMS_FEATURES + ['item_id']]

item_encoders = {}
for feat in ITEMS_FEATURES:
    le = LabelEncoder()
    items_df[feat] = le.fit_transform(items_df[feat])
    item_encoders[feat] = le

items_df.head()

,content_type,release_year,for_kids,age_rating,studios,countries,directors,item_id
0,0,86,0,4,33,258,5671,10711
1,0,98,0,4,33,421,6546,2508
2,0,95,0,4,33,298,95,10716
3,0,99,0,4,33,57,7735,7868
4,0,62,0,3,34,419,1544,16268


Для каждой фичи пропишем размерность эмбедингов

In [7]:
items_categorical_size = {feat: items_df[feat].nunique() for feat in ITEMS_FEATURES}
items_features_info = [
    (items_categorical_size['content_type'], 8),
    (items_categorical_size['release_year'], 8),
    (items_categorical_size['for_kids'], 2),
    (items_categorical_size['age_rating'], 8),
    (items_categorical_size['studios'], 16),
    (items_categorical_size['countries'], 8),
    (items_categorical_size['directors'], 16),
]

In [8]:
users_categorical_size = {feat: users_df[feat].nunique() for feat in USERS_FEATURES}
users_features_info = [
    (users_categorical_size['age'], 16),
    (users_categorical_size['income'], 16),
    (users_categorical_size['sex'], 2),
    (users_categorical_size['kids_flg'], 2),
]

# Пре-процессинг
В этой секции происходят базовые преобразования, как было показано на лекции. Исключением лишь является то, что фичи пользователей и айтемов не преобразовываются как one-hot вектора, потому что я буду использовать
обучаемые эмбеддинги

In [9]:
# Input:
# - interactions_df — сырые взаимодействия пользователей с фильмами

# Output:
# - Очищенный interactions_df, содержащий:
#   - Только пользователей, просмотревших более 10 фильмов.
#   - Только фильмы, просмотренные более чем 10 пользователями.
#   - Только взаимодействия, где просмотр фильма превышает 10%.

# Преобразования:
# - Удалены взаимодействия с watched_pct ≤ 10%.
# - Оставлены только активные пользователи (с более чем 10 взаимодействиями).
# - Оставлены только популярные фильмы (с более чем 10 пользователями).
# - Итоговый датасет уменьшен в объеме, но очищен для повышения качества обучения модели.


interactions_df = interactions_df[interactions_df.watched_pct > 10]
valid_users = []
c = Counter(interactions_df.user_id)
for user_id, entries in c.most_common():
    if entries > 10:
        valid_users.append(user_id)
valid_items = []
c = Counter(interactions_df.item_id)
for item_id, entries in c.most_common():
    if entries > 10:
        valid_items.append(item_id)

interactions_df = interactions_df[interactions_df.user_id.isin(valid_users)]
interactions_df = interactions_df[interactions_df.item_id.isin(valid_items)]

In [10]:
# Input:
# - interactions_df — очищенные взаимодействия пользователей и фильмов.
# - users_df — таблица с пользователями и их признаками (НЕ one-hot).
# - items_df — таблица с фильмами и их признаками (НЕ one-hot).

# Output:
# - interactions_df, items_df и users_df — синхронизированные таблицы, содержащие только общих пользователей и фильмы.

# Преобразования:
# - Найдены пересечения пользователей и фильмов, присутствующих одновременно в interactions_df и users_df/items_df.
# - Удалены пользователи и фильмы, отсутствующие в обоих соответствующих датасетах.
# - Гарантирована консистентность между взаимодействиями и признаковыми таблицами (users_df и items_df).


common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6901


In [11]:
common_users = set(interactions_df.user_id.unique()).intersection(set(users_df.user_id.unique()))
common_items = set(interactions_df.item_id.unique()).intersection(set(items_df.item_id.unique()))

print(len(common_users))
print(len(common_items))

interactions_df = interactions_df[interactions_df.item_id.isin(common_items)]
interactions_df = interactions_df[interactions_df.user_id.isin(common_users)]

items_df = items_df[items_df.item_id.isin(common_items)]
users_df = users_df[users_df.user_id.isin(common_users)]

65974
6897


# Пост-процессинг : train и test

### Временной сплит для построения interaction матриц:
- Делим interactions_df по дате на train/test по последним N дней.
- Кодируем user_id и item_id в train через категориальные uid и iid.
- Применяем тот же маппинг к test.
- Строим interaction матрицы отдельно для train и test.
- Таким образом, обучаем модель на истории, а тестируем на предсказании будущих интересов пользователя.


In [12]:
# Разбиваем датасет на трейн и тестовую часть
N_DAYS = 7
max_date = interactions_df['last_watch_dt'].max()

train_df = interactions_df[interactions_df['last_watch_dt'] <= max_date - pd.Timedelta(days=N_DAYS)]
test_df = interactions_df[interactions_df['last_watch_dt'] > max_date - pd.Timedelta(days=N_DAYS)]


# Только юзеры, которые были в train
test_df = test_df[test_df['user_id'].isin(train_df['user_id'])]
# Только айтемы, которые были в train
test_df = test_df[test_df['item_id'].isin(train_df['item_id'])]

In [13]:

# категориально закодируем user_id и item_id по train
train_df['uid'] = train_df['user_id'].astype('category').cat.codes
train_df['iid'] = train_df['item_id'].astype('category').cat.codes

# Сделаем маппинг user_id и item_id в uid и iid

uid_to_user_id = dict(zip(train_df['uid'], train_df['user_id']))
iid_to_item_id = dict(zip(train_df['iid'], train_df['item_id']))

user_id_to_uid = dict(zip(train_df['user_id'], train_df['uid']))
item_id_to_iid = dict(zip(train_df['item_id'], train_df['iid']))

# Создадим колонки uid и iid в test_df и применим маппинг

test_df['uid'] = test_df['user_id'].map(user_id_to_uid)
test_df['iid'] = test_df['item_id'].map(item_id_to_iid)

print(f"Test: {test_df.shape}")
print(f"Train: {train_df.shape}")

Test: (83707, 7)
Train: (1375329, 7)


In [14]:
# Оставим только те айтемы которые есть в train

items_df = items_df[items_df['item_id'].isin(train_df['item_id'])]
items_df = items_df.set_index('item_id').loc[train_df['item_id'].unique()].reset_index()

# Убедимся, что количество уникальных айтомов в items_df совпадает с количеством уникальных айтемов в train_df
assert items_df.item_id.nunique() == train_df.item_id.nunique()

In [15]:
# Оставим только тех юзеров которые есть в train

users_df = users_df[users_df['user_id'].isin(train_df['user_id'])]
users_df = users_df.set_index('user_id').loc[train_df['user_id'].unique()].reset_index()

# Убедимся, что количество уникальных юзеров в users_df совпадает с количеством уникальных юзеров в train_df
assert users_df.user_id.nunique() == train_df.user_id.nunique()

In [16]:
# Проверим, что маппинг был корректный
assert test_df[test_df['uid'] == 4375].user_id.values[0] == train_df[train_df['uid'] == 4375].user_id.values[0]

Сделаем матрицу интеракция для train и test. Размерность одинаковая

In [17]:
import numpy as np

n_users = train_df['uid'].nunique()
n_items = train_df['iid'].nunique()

# ==========================

train_vec = np.zeros((n_users, n_items))
for uid, iid in zip(train_df['uid'], train_df['iid']):
    train_vec[uid, iid] += 1

# нормализация
train_vec = train_vec / train_vec.sum(axis=1, keepdims=True)

# ==========================

# для test
test_vec = np.zeros((n_users, n_items))
for uid, iid in zip(test_df['uid'], test_df['iid']):
    test_vec[uid, iid] += 1

test_vec = test_vec / test_vec.sum(axis=1, keepdims=True)

In [18]:
print(f"Train interaction matrix shape : {train_vec.shape}")
print(f"Test interaction matrix shape : {test_vec.shape}")

Train interaction matrix shape : (65792, 6862)
Test interaction matrix shape : (65792, 6862)


In [20]:
print("Train: Размер уникальных юзеров", train_df.uid.nunique())
print("Test: Размер уникальных юзеров", test_df.uid.nunique())


Train: Размер уникальных юзеров 65792
Test: Размер уникальных юзеров 25676


In [21]:
N_FACTORS = 128

ITEM_MODEL_SHAPE = (items_df.drop(["item_id"], axis=1).shape[1], )
USER_META_MODEL_SHAPE = (users_df.drop(["user_id"], axis=1).shape[1], )

USER_INTERACTION_MODEL_SHAPE = (train_vec.shape[1], )

print(f"N_FACTORS: {N_FACTORS}")
print(f"ITEM_MODEL_SHAPE: {ITEM_MODEL_SHAPE}")
print(f"USER_META_MODEL_SHAPE: {USER_META_MODEL_SHAPE}")
print(f"USER_INTERACTION_MODEL_SHAPE: {USER_INTERACTION_MODEL_SHAPE}")

N_FACTORS: 128
ITEM_MODEL_SHAPE: (7,)
USER_META_MODEL_SHAPE: (4,)
USER_INTERACTION_MODEL_SHAPE: (6862,)
